In [1]:
!pip install gdown
!pip install transformers accelerate pyiqa -U
!gdown 1qAuSSzPUXz5jgAz3xzQ0IGciK5Aqilnm

Downloading...
From: https://drive.google.com/uc?id=1qAuSSzPUXz5jgAz3xzQ0IGciK5Aqilnm
To: /kaggle/working/prompts_generated.json
100%|███████████████████████████████████████| 10.6M/10.6M [00:00<00:00, 141MB/s]


In [51]:
import json
import pandas as pd
import os

with open('prompts_generated.json', 'r') as f:
    data = json.loads(f.read())

keys = ['caption', 'image_photo_name', 'image_sticker_mixed_name', 'image_sticker_name', 'prompt_sticker', 'prompt']

d = {i:[] for i in keys}

for _, val in tqdm(data.items(), total=len(data)):
    fnames = ['image_photo_name', 'image_sticker_mixed_name', 'image_sticker_name']
    stop = False 
    for fname in fnames:
        val[fname] = os.path.join('/kaggle/input/synthetic-stickers-dataset/imgs', val[fname])
        if not os.path.exists(val[fname]):
            stop = True
            break
            
    if stop:
        continue
            
    if not(val['prompt_sticker'].startswith('A sticker of') and val['prompt_sticker'].endswith(', cartoon style.')):
        continue
    
    
    
    for key in keys:
        if key in ['prompt_sticker', 'prompt']:
            continue
        d[key].append(val[key])
        
    p = val['prompt_sticker']
    p = p[:len(p) - 16] + '.'
    d['prompt_sticker'].append(p)
    d['prompt'].append(p.replace('A sticker', 'A photo', 1))
        

df = pd.DataFrame(d)
df

  0%|          | 0/15211 [00:00<?, ?it/s]

,caption,image_photo_name,image_sticker_mixed_name,image_sticker_name,prompt_sticker,prompt
0,the image shows an old television with a black...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a classic black and white tv.,A photo of a classic black and white tv.
1,"tranquil canal scene with colorful buildings, ...",/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a tranquil venice canal.,A photo of a tranquil venice canal.
2,"blues artist, wearing a tan hat and khaki pant...",/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a soulful american blues guitarist.,A photo of a soulful american blues guitarist.
3,"amidst a serene blue sky, a tall green tree st...",/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a tranquil pond with water lilies.,A photo of a tranquil pond with water lilies.
4,"an adult kangaroo with pointy ears, black nose...",/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a cute baby kangaroo.,A photo of a cute baby kangaroo.
...,...,...,...,...,...,...
15186,the image features a festive table set with a ...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a festive new years eve celebrati...,A photo of a festive new years eve celebration...
15187,get ready for a festive christmas party with a...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a festive mardi gras celebration ...,A photo of a festive mardi gras celebration wi...
15188,the image features a colorful assortment of do...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a rainbow-colored macaron tower t...,A photo of a rainbow-colored macaron tower top...
15189,"grilled corn on the cob and corn on the cob, a...",/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,/kaggle/input/synthetic-stickers-dataset/imgs/...,A sticker of a farm-fresh ear of corn on the c...,A photo of a farm-fresh ear of corn on the cob...


In [21]:
from scipy.spatial.qhull import QhullError
from scipy import spatial
spatial.QhullError = QhullError
from tqdm.auto import tqdm
import io
import glob
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import pyiqa
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import zlib
import requests
from transformers import AutoProcessor, AutoModel
import threading
import os
import shutil
import pyiqa
from torchvision.transforms.functional import pil_to_tensor
import cv2
from transformers import AutoImageProcessor
import numpy as np

/tmp/ipykernel_33/696022271.py:1: DeprecationWarning: Please import `QhullError` from the `scipy.spatial` namespace; the `scipy.spatial.qhull` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.spatial.qhull import QhullError
2024-03-10 01:46:48.184260: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 01:46:48.184383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 01:46:48.323142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [23]:
clip_model_id = 'google/siglip-so400m-patch14-384'
iqa_model_id = 'topiq_iaa'
device = 'cuda:0'
SIZE = 512


aes_metric = pyiqa.create_metric(iqa_model_id).net.to('cuda:0')

clip_model = AutoModel.from_pretrained(clip_model_id).to('cuda:0')
processor = AutoProcessor.from_pretrained(clip_model_id)



IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)])

Loading pretrained model CFANet from /root/.cache/torch/hub/checkpoints/cfanet_iaa_ava_swin-393b41b4.pth


2024-03-10 01:50:55,981 INFO: Metric [CFANet] is created.


In [66]:
def cosine_sim(x, y):
    return torch.nn.functional.cosine_similarity(x, y)

def aesthetic_score(x, device):
    x = transform(x).to(device)
    return aes_metric(x) / 10

def get_metrics(img_orig, img_sticker, prompt_sticker, prompt, device='cuda:0'):
    img_orig_preprocessed = processor(images=img_orig, return_tensors="pt").to(device)
    img_sticker_preprocessed = processor(images=img_sticker, return_tensors="pt").to(device)

    prompt_is_sticker_preprocessed = processor(text='a sticker.', return_tensors="pt", padding="max_length").to(device)
    prompt_sticker_preprocessed = processor(text=prompt_sticker, return_tensors="pt", padding="max_length").to(device)
    prompt_preprocessed = processor(text=prompt, return_tensors="pt", padding="max_length").to(device)

    with torch.no_grad():
        img_orig_emb = clip_model.get_image_features(**img_orig_preprocessed).to(device)
        img_sticker_emb = clip_model.get_image_features(**img_sticker_preprocessed).to(device)

        prompt_is_sticker_emb = clip_model.get_text_features(**prompt_is_sticker_preprocessed).to(device)
        prompt_sticker_emb = clip_model.get_text_features(**prompt_sticker_preprocessed).to(device)
        prompt_emb = clip_model.get_text_features(**prompt_preprocessed).to(device)

    x1 = cosine_sim(img_orig_emb, img_sticker_emb).cpu().detach().numpy()[0]
    x2 = cosine_sim(img_orig_emb, prompt_emb).cpu().detach().numpy()[0]
    x3 = aesthetic_score(img_sticker, device).cpu().detach().numpy()[0][0]
    x4 = cosine_sim(img_sticker_emb, prompt_is_sticker_emb).cpu().detach().numpy()[0]
    x5 = cosine_sim(img_sticker_emb, prompt_sticker_emb).cpu().detach().numpy()[0]
    x6 = aesthetic_score(img_orig, device).cpu().detach().numpy()[0][0]

    data = {}

    data['sim_between_real_img_and_generated_sticker'] = x1
    data['sim_between_real_img_and_its_prompt'] = x2
    data['aes_img'] = x6

    data['sim_between_generated_sticker_and_is_sticker_prompt'] = x4
    data['sim_between_generated_sticker_and_its_prompt'] = x5
    data['aes_sticker'] = x3
    return data


def parse_metrics(df, _type=1):
    full_keys = ['caption', 'image_photo_name',
                 'image_sticker_name', 'prompt_sticker', 'prompt', 'type', 'sim_between_real_img_and_generated_sticker',
                 'sim_between_real_img_and_its_prompt', 'aes_img',
                 'sim_between_generated_sticker_and_is_sticker_prompt', 'sim_between_generated_sticker_and_its_prompt',
                 'aes_sticker']
    keys_copy = ['caption', 'image_photo_name', 'prompt_sticker', 'prompt']
    data = {i: [] for i in full_keys}

    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        img_orig, prompt_sticker, prompt = Image.open(
            row['image_photo_name']), row['prompt_sticker'], row['prompt']
        img_sticker = Image.open(row['image_sticker_name']) if _type == 1 else Image.open(
            row['image_sticker_mixed_name'])
        metrics = get_metrics(img_orig, img_sticker, prompt_sticker, prompt)
        if _type == 1:
            data['image_sticker_name'].append(row['image_sticker_name'])
            data['type'].append(1)
        else:
            data['image_sticker_name'].append(row['image_sticker_mixed_name'])
            data['type'].append(2)

        for key, val in metrics.items():
            data[key].append(val)

        for i in keys_copy:
            data[i].append(row[i])
    return pd.DataFrame(data)


In [70]:
df1 = parse_metrics(df)
df2 = parse_metrics(df, _type=2)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
df1.to_csv('df1.csv', index=False)
df2.to_csv('df2.csv', index=False)